In [1]:
!echo $CONDA_DEFAULT_ENV

fairseq_hcg_mha


In [2]:
import sys

fairseq_path = '/data_new/d.tarasov/workspace/fairseq'
if fairseq_path not in sys.path:
    sys.path = [ fairseq_path ] + sys.path

sys.path

['/data_new/d.tarasov/workspace/fairseq',
 '/home/dyutarasov/workspace/fairseq',
 '/opt/software/python/anaconda/2019_10/lib/python37.zip',
 '/opt/software/python/anaconda/2019_10/lib/python3.7',
 '/opt/software/python/anaconda/2019_10/lib/python3.7/lib-dynload',
 '',
 '/opt/software/python/anaconda/2019_10/lib/python3.7/site-packages',
 '/opt/software/python/anaconda/2019_10/lib/python3.7/site-packages/IPython/extensions',
 '/home/dyutarasov/.ipython']

In [ ]:
!ls -l checkpoints_nonautoregressive_transformer_hcg_pruning_lambda_0.1/

In [4]:
from fairseq import checkpoint_utils, options, tasks, utils

ModuleNotFoundError: No module named 'omegaconf'

In [9]:
checkpoint_path = 'checkpoints_nonautoregressive_transformer_hcg_pruning_lambda_0/checkpoint_last.pt'
model = checkpoint_utils.load_model_ensemble(utils.split_paths( 'checkpoints_max_tokens_4k/checkpoint_best.pt' ))
model

([NATransformerModel(
    (encoder): FairseqNATEncoder(
      (dropout_module): FairseqDropout()
      (embed_tokens): Embedding(39840, 512, padding_idx=1)
      (embed_positions): LearnedPositionalEmbedding(1026, 512, padding_idx=1)
      (layers): ModuleList(
        (0): TransformerEncoderLayerBase(
          (self_attn): MultiheadAttention(
            (dropout_module): FairseqDropout()
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (dropout_module): FairseqDropout()
          (activation_dropout_module): FairseqDropout()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_featu

In [7]:
type(model)

dict

In [21]:
import torch
torch.repeat_interleave( torch.unsqueeze(torch.zeros((14, 160)), -1), 160, dim=2 ).shape

torch.Size([14, 160, 160])

In [24]:
from fairseq.modules import MultiheadAttention
from fairseq.modules.hcg_attention import MultiHeadHCGAttention

In [37]:
def test_mha(mha_class):
    
    hidd_dim = 64
    num_heads = 8

    mha = mha_class(hidd_dim, num_heads)

    qkv = torch.rand((7, 8, hidd_dim))
    att_res, _ = mha.forward(qkv, qkv, qkv)
    
    att_res.shape == qkv.shape
    
    return att_res

In [ ]:
test_mha(MultiHeadHCGAttention)

In [41]:
def count_trainable_params(module):
    return sum( p.numel() for p in module.parameters() if p.requires_grad )

In [43]:
mha_hid_dim = 64
mha_num_heads = 8

mha = MultiheadAttention(mha_hid_dim, mha_num_heads)
mha.dropout_module.p = 0
count_trainable_params(mha)

16640

In [58]:
from collections import Counter

In [59]:
Counter( p.shape for p in mha.parameters() if p.requires_grad )

Counter({torch.Size([64, 64]): 4, torch.Size([64]): 4})

In [60]:
Counter( p.shape for p in hcg_mha.parameters() if p.requires_grad )

Counter({torch.Size([8, 64]): 24,
         torch.Size([8]): 24,
         torch.Size([64, 64]): 1,
         torch.Size([64]): 1})

In [64]:
mha

MultiheadAttention(
  (dropout_module): FairseqDropout()
  (k_proj): Linear(in_features=64, out_features=64, bias=True)
  (v_proj): Linear(in_features=64, out_features=64, bias=True)
  (q_proj): Linear(in_features=64, out_features=64, bias=True)
  (out_proj): Linear(in_features=64, out_features=64, bias=True)
)

In [63]:
hcg_mha

MultiHeadHCGAttention(
  (attention_heads): ModuleList(
    (0): HCGAttention(
      (query_weights): Linear(in_features=64, out_features=8, bias=True)
      (key_weights): Linear(in_features=64, out_features=8, bias=True)
      (value_weights): Linear(in_features=64, out_features=8, bias=True)
      (softmax): Softmax(dim=-1)
    )
    (1): HCGAttention(
      (query_weights): Linear(in_features=64, out_features=8, bias=True)
      (key_weights): Linear(in_features=64, out_features=8, bias=True)
      (value_weights): Linear(in_features=64, out_features=8, bias=True)
      (softmax): Softmax(dim=-1)
    )
    (2): HCGAttention(
      (query_weights): Linear(in_features=64, out_features=8, bias=True)
      (key_weights): Linear(in_features=64, out_features=8, bias=True)
      (value_weights): Linear(in_features=64, out_features=8, bias=True)
      (softmax): Softmax(dim=-1)
    )
    (3): HCGAttention(
      (query_weights): Linear(in_features=64, out_features=8, bias=True)
      (key_

In [45]:
hcg_mha = MultiHeadHCGAttention(mha_hid_dim, mha_num_heads)
count_trainable_params(hcg_mha)

16640

In [46]:
hcg_mha

MultiHeadHCGAttention(
  (attention_heads): ModuleList(
    (0): HCGAttention(
      (query_weights): Linear(in_features=64, out_features=8, bias=True)
      (key_weights): Linear(in_features=64, out_features=8, bias=True)
      (value_weights): Linear(in_features=64, out_features=8, bias=True)
      (softmax): Softmax(dim=-1)
    )
    (1): HCGAttention(
      (query_weights): Linear(in_features=64, out_features=8, bias=True)
      (key_weights): Linear(in_features=64, out_features=8, bias=True)
      (value_weights): Linear(in_features=64, out_features=8, bias=True)
      (softmax): Softmax(dim=-1)
    )
    (2): HCGAttention(
      (query_weights): Linear(in_features=64, out_features=8, bias=True)
      (key_weights): Linear(in_features=64, out_features=8, bias=True)
      (value_weights): Linear(in_features=64, out_features=8, bias=True)
      (softmax): Softmax(dim=-1)
    )
    (3): HCGAttention(
      (query_weights): Linear(in_features=64, out_features=8, bias=True)
      (key_

In [33]:
test_mha(MultiheadAttention)

tensor([[[-0.4929, -0.2564,  0.0943,  ...,  0.3369,  0.5001,  0.3237],
         [-0.5830, -0.2702,  0.3887,  ...,  0.2738,  0.6114,  0.3171],
         [-0.5882, -0.1335,  0.3095,  ...,  0.2822,  0.6535,  0.1725],
         ...,
         [-0.5204, -0.2848,  0.3575,  ...,  0.1799,  0.4535,  0.1451],
         [-0.6387, -0.1075,  0.4367,  ...,  0.4107,  0.5984,  0.1606],
         [-0.3994, -0.2388,  0.3155,  ...,  0.2539,  0.5598,  0.0820]],

        [[-0.4919, -0.2583,  0.0970,  ...,  0.3367,  0.4935,  0.3277],
         [-0.5808, -0.2730,  0.3884,  ...,  0.2778,  0.6175,  0.3167],
         [-0.5850, -0.1261,  0.3159,  ...,  0.2793,  0.6603,  0.1727],
         ...,
         [-0.5182, -0.2839,  0.3581,  ...,  0.1781,  0.4549,  0.1379],
         [-0.6355, -0.1100,  0.4374,  ...,  0.4108,  0.5951,  0.1594],
         [-0.3985, -0.2439,  0.3257,  ...,  0.2502,  0.5583,  0.0751]],

        [[-0.4975, -0.2561,  0.0931,  ...,  0.3369,  0.4989,  0.3251],
         [-0.5760, -0.2723,  0.3966,  ...,  0

In [72]:
a, b = torch.rand((3, 64, 8)), torch.rand((3, 8, 64))


torch.bmm(a, b) == torch.matmul(a, b)

tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

        [[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

        [[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]]]